In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("../datasets/soc-Epinions1.txt", sep='\t', header=4)
data = data.as_matrix()

In [59]:
#создание epinion удалив от туда пользователей, у которых количество отзывов меньше 25

In [4]:
list_good_user = list(np.where(np.bincount(data[:, 0]) > 24)[0])
set_good_user = set(list_good_user)
epinion = []
for i,j in data:
    if i in set_good_user:
        epinion.append([i, j])
epinion = np.array(epinion)
for new_u, u in enumerate(list_good_user):
    epinion[epinion[:, 0] == u, 0] = new_u

In [6]:
def givenK_train_test(data, K):
    xlist = []
    for u in np.unique(data[:, 0]):
        xlist.append(data[data[:, 0] == u,  1])
    
    
    train = []
    test = []
    for x in xlist:
        train_ind = np.random.choice(x.shape[0], K, replace=False)
        tmp = np.ones(x.shape[0])
        tmp[train_ind] = 0
        test_ind = np.arange(x.shape[0])[tmp == 1]
        
        train.append(x[train_ind])
        test.append(x[test_ind])
        
    return (train, test, xlist)

In [107]:
def get_prec_K(K, train, test,  cls,  skip_train=False):
    res = 0
    for u, (test_cur, train_cur) in enumerate(zip(test, train)):
        ulist = cls.get_list(u)
        if  skip_train:
            mask = np.in1d(ulist, train_cur)
            ulist = ulist[~mask]
        
        mask = np.in1d(ulist[:K], test_cur)
        if not skip_train:
            mask2 = np.in1d(ulist[:K], train_cur)
            mask = np.logical_or(mask, mask2)
        
        if u == 0:
            print(test_cur)
            print(train_cur)
            print(ulist[:K + train_cur.shape[0]])
            print(mask)
            print(np.mean(mask))
        
        res += np.mean(mask)
        
    res /= len(test)
    return res


def get_one_recal_K(K, train, test,  cls,  skip_train=False):
    res = 0
    for u, (test_cur, train_cur) in enumerate(zip(test, train)):
        ulist = cls.get_list(u)
        if  skip_train:
            mask = np.in1d(ulist, train_cur)
            ulist = ulist[~mask]
        mask = np.in1d(ulist[:K], test_cur)
        if not skip_train:
            mask2 = np.in1d(ulist[:K], train_cur)
            mask = np.logical_or(mask, mask2)
        res += int(True in mask)
        
    res /= len(test)
    return res


def get_RR(train, test, ulist, skip_train=False):
    for rank, item in enumerate(ulist):
        if item in test:
            return 1 / (rank + 1)
        if not skip_train and item in train:
            return 1 / (rank  + 1)
    return 1/ulist.shape[0]


def get_MRR(train, test,  cls, skip_train=False):
    res = 0
    for u, (test_cur, train_cur) in enumerate(zip(test, train)):
        ulist = cls.get_list(u)
        res += get_RR(train_cur, test_cur, ulist, skip_train)
    res /= len(test)
    return res
        

In [108]:
class PopRec:
    def __init__(self):
        pass

    
    def fit(self, data):
        votes = np.bincount(np.array(data).ravel())
        self.most_pop = np.argsort(-votes)
        
        
    def get_list(self, u):
        return self.most_pop

In [97]:
train, test, traintest = givenK_train_test(epinion, 5)

In [98]:
rankmethod = PopRec()
rankmethod.fit(train)

In [99]:
print(get_prec_K(5, train, test, rankmethod, skip_train=False))

[  5   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23
  24  25  26  28  29  30  31  32  33  34  35  36  37  38  39  40  42  43
  44  45  46  47  48  49  50  51  52  53  54  55  56  58  59  60  61  62
  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80
  81  82  83  84  85  86  88  89  90  91  92  93  94  95  96  97  98  99
 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 131 132 133 134 135 136
 137 138 139 140 141 142 799]
[ 87  41  57 130  27]
[  18  136  143 1179  790  737 1719  118   34   40]
[ True  True False False False]
0.4
0.209692206942


In [100]:
print(get_prec_K(5, train, test, rankmethod, skip_train=True))

[  5   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23
  24  25  26  28  29  30  31  32  33  34  35  36  37  38  39  40  42  43
  44  45  46  47  48  49  50  51  52  53  54  55  56  58  59  60  61  62
  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80
  81  82  83  84  85  86  88  89  90  91  92  93  94  95  96  97  98  99
 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 131 132 133 134 135 136
 137 138 139 140 141 142 799]
[ 87  41  57 130  27]
[  18  136  143 1179  790  737 1719  118   34   40]
[ True  True False False False]
0.4
0.195328530888


In [101]:
print(get_one_recal_K(5, train, test, rankmethod, skip_train=False))

0.5701811831477843


In [104]:
print(get_one_recal_K(5, train, test, rankmethod, skip_train=True))

0.5444226151495307


In [105]:
print(get_MRR(train, test, rankmethod, skip_train=False))

0.4551690233053217


In [109]:
print(get_MRR(train, test, rankmethod, skip_train=True))

0.42663367860506773


In [65]:
mask = np.array([True, True])
int(True in mask)
 

1

In [73]:
a = np.array([1, 3, 5, 2,9, 1])
np.argsort(-a)[:5]

np.repeat(np.array([a]), 5, axis=0)

array([[1, 3, 5, 2, 9, 1],
       [1, 3, 5, 2, 9, 1],
       [1, 3, 5, 2, 9, 1],
       [1, 3, 5, 2, 9, 1],
       [1, 3, 5, 2, 9, 1]])

In [18]:
a = np.bincount(data.ix[:, 0])

In [27]:
np.where(a < 24)

(array([    7,     9,    14, ..., 75885, 75886, 75887]),)

In [26]:
a.shape[0] - np.sum(a < 24)

4720